In [1]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import optuna
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')  # ignore notifications
import sys
import os


In [2]:
import pandas as pd
df = pd.read_excel('COM.xlsx')
pd.set_option('display.max_columns', None)
df.head()

,listing litle,city,area,location,zone,location_hub,property_type,ownership,size_in_sqft,carpet_area_sqft,private_washroom,public_washroom,floor_no,total_floors,amenities_count,electric_charge_included,water_charge_included,property_age,possession_status,posted_by,lock in period,expected rent increases yearly,negotiable,brokerage,security_deposite,rent_price
0,showroom for rent,nagpur,dharampeth,"dharampeth,nagpur",east,Retail Complex/ Building,showroom,freehold,900 sqft,750 sqft,1,1,1 floor,1 floor,"CCTV,food court(2)",no,yes,5 years,ready to move,housing expert,6 months,0.10,yes,yes,450000.0,75000
1,ready to use office space,nagpur,ramdaspeth,"ramdaspeth, nagpur",east,business park,ready to use office,freehold,600 sqft,500 sqft,1,1,1 floor,1 floor,"lift, cabin,metting room(3)",no,yes,6 years,ready to move,housing expert,2 months,0.05,yes,yes,NaN,15000
2,shop for rent,nagpur,manewada,"manewada, nagpur",south,others,shop,freehold,300 sqft,250 sqft,1,0,ground floor,2 floors,0,no,yes,NaN,ready to move,owner,12 months,0.10,yes,no,25000.0,12000
3,commercial property,nagpur,manewada,"manewada, nagpur",south,others,commercial property,freehold,1350 sqft,1012 sqft,1,1,ground floor,2 floors,0,yes,yes,5 years,ready to move,housing expert,6 months,0.10,yes,yes,40000.0,20000
4,ready to use office space,nagpur,manewada,"vittal nagar ,manewda,nagpur",south,others,ready to use office,freehold,2000 sqft,1800 sqft,1,1,ground floor,2 floors,"water storage, internet,cabin,meeting rooms(4)",yes,yes,10 years,ready to move,owner,6 months,NaN,no,no,200000.0,100000


In [3]:
for col in df.columns:
    print(f"Value counts for column: {col}")
    print(df[col].value_counts())
    print("-" * 40)

Value counts for column: listing litle
listing litle
showroom for rent            243
shop for rent                216
bare shell office space      211
ready to use office space    171
commercial property           84
werehouse                     60
godown for rent               20
Name: count, dtype: int64
----------------------------------------
Value counts for column: city
city
nagpur    1005
Name: count, dtype: int64
----------------------------------------
Value counts for column: area
area
manewada             73
jaitala              71
besa                 67
omkar nagar          59
itwari               56
hingna               52
sitabuldi            52
mahal                47
kharbi               44
pratap nagar         36
mihan                36
ramdaspeth           33
dharampeth           32
gandhibag            31
chatrapati nagar     29
nandanwan            23
sadar                21
somalwada            19
dighori              19
ganeshpeth colony    18
sakkardara       

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   listing litle                   1005 non-null   object 
 1   city                            1005 non-null   object 
 2   area                            1005 non-null   object 
 3   location                        1005 non-null   object 
 4   zone                            1005 non-null   object 
 5   location_hub                    1005 non-null   object 
 6   property_type                   1005 non-null   object 
 7   ownership                       1005 non-null   object 
 8   size_in_sqft                    1005 non-null   object 
 9   carpet_area_sqft                786 non-null    object 
 10  private_washroom                1005 non-null   int64  
 11  public_washroom                 1005 non-null   int64  
 12  floor_no                        10

In [5]:
df = df.drop('location', axis=1)

In [6]:
df = df.replace(" ", 0).fillna(0)

In [7]:
import pandas as pd
import re

# Function to convert floor string into a list of floor numbers
def floor_to_int_list(floor_str):
    floor_str = str(floor_str).lower()
    floor_str = floor_str.replace('floor', '').replace('floors', '').replace(' ', '')
    # Replace ground floor / GF with 0
    floor_str = floor_str.replace('ground', '0').replace('gf', '0')
    # Split multiple floors
    parts = re.split(r'[,]', floor_str)
    floor_numbers = []
    for p in parts:
        try:
            floor_numbers.append(int(p))
        except:
            continue
    return sorted(set(floor_numbers)) if floor_numbers else []

# Apply transformation
df['floor_list'] = df['floor_no'].apply(floor_to_int_list)

# Create 10 floor columns (0–9)
for i in range(10):
    df[f'floor_{i}'] = df['floor_list'].apply(lambda x: 1 if i in x else 0)

# Drop helper column if not needed
df.drop(columns=['floor_list'], inplace=True)

print(df.head(20))


                listing litle    city            area   zone  \
0           showroom for rent  nagpur      dharampeth   east   
1   ready to use office space  nagpur      ramdaspeth   east   
2               shop for rent  nagpur        manewada  south   
3         commercial property  nagpur        manewada  south   
4   ready to use office space  nagpur        manewada  south   
5               shop for rent  nagpur        manewada  south   
6         commercial property  nagpur  trimurti nagar  south   
7           showroom for rent  nagpur       somalwada  south   
8           showroom for rent  nagpur     civil lines   east   
9           showroom for rent  nagpur      laxminagar   west   
10          showroom for rent  nagpur       bagadganj   east   
11          showroom for rent  nagpur           pardi  south   
12          showroom for rent  nagpur          itwari   west   
13          showroom for rent  nagpur          itwari   west   
14          showroom for rent  nagpur   

In [8]:
import pandas as pd

def total_floors_to_int(floor_str):
    try:
        # Remove 'floors' or 'floor' and spaces, then convert to int
        return int(str(floor_str).lower().replace('floors', '').replace('floor', '').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['total_floors'] = df['total_floors'].apply(total_floors_to_int)




In [9]:
import pandas as pd

# Function to convert size to integer
def size_to_int(size_str):
    try:
        # Remove 'sqft' or 'sq.ft' and spaces, then convert to int
        return int(str(size_str).lower().replace('sqft','').replace('sq.ft','').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['size_in_sqft'] = df['size_in_sqft'].apply(size_to_int)

# Check result
print(df[['size_in_sqft']].head(20))


    size_in_sqft
0            900
1            600
2            300
3           1350
4           2000
5            200
6           6000
7            550
8           3500
9           1300
10          2600
11          1600
12          1000
13           550
14          2100
15           950
16          1000
17          1200
18          1600
19           250


In [10]:
import pandas as pd

# Function to convert size to integer
def size_to_int(size_str):
    try:
        # Remove 'sqft' or 'sq.ft' and spaces, then convert to int
        return int(str(size_str).lower().replace('sqft','').replace('sq.ft','').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['carpet_area_sqft'] = df['carpet_area_sqft'].apply(size_to_int)

# Check result
print(df[['carpet_area_sqft']].head(20))

    carpet_area_sqft
0                750
1                500
2                250
3               1012
4               1800
5                175
6               4000
7                500
8               3200
9               1250
10              2600
11              1500
12               900
13               550
14              1900
15               800
16              1000
17              1152
18              1600
19               150


In [11]:
print(df['amenities_count'].value_counts())

amenities_count
parking, vastu(2)                                                                                         34
lift, cabin,metting room(3)                                                                               30
parking(1)                                                                                                21
DG and UPS , parking, staircases, CCTV, power backup, reception area, pantry (8)                          21
parking, water storae, CCTV, power backup, internet, reception area, fire sensors, furnishing,lift (9)    21
                                                                                                          ..
water storage, CCTV, paarking, vastu(4)                                                                    1
security, parking, CCTV(2)                                                                                 1
parking, CCTV , security, water storage(4)                                                                 1
CCT

In [12]:
# Function to extract amenities as a list
def extract_amenities_list(text):
    text = str(text).lower()
    # Remove counts in parentheses
    text = re.sub(r'\(\d+\)', '', text)
    # Split by comma and strip spaces
    amenities = [x.strip() for x in text.split(',') if x.strip() != '']
    return amenities

# Apply function
df['amenities_list'] = df['amenities_count'].apply(extract_amenities_list)

# Get unique amenities across all rows
all_amenities = set([amenity for sublist in df['amenities_list'] for amenity in sublist])

# Create binary columns
for amenity in all_amenities:
    df[amenity.replace(' ', '_')] = df['amenities_list'].apply(lambda x: 1 if amenity in x else 0)

# Drop intermediate column if needed
df = df.drop(columns=['amenities_list'])


In [13]:

import re

def extract_amenities_list(text):
    text = str(text).lower()
    # Remove counts in parentheses
    text = re.sub(r'\(\d+\)', '', text)
    # Split by comma and strip spaces
    amenities = [x.strip() for x in text.split(',') if x.strip() != '']
    return amenities

# Create a list of all amenities
all_amenities = set([amenity for sublist in df['amenities_count'].apply(extract_amenities_list) for amenity in sublist])

# Optionally, replace spaces with underscores for column names
binary_columns = [amenity.replace(' ', '_') for amenity in sorted(all_amenities)]

print(binary_columns)


['0', 'cabin', 'cctv', 'ccvt', 'cebin', 'dg', 'dg_and_ups', 'fire_extinghuishers', 'fire_extinguisher', 'fire_extinguishers', 'fire_safety', 'fire_safty', 'fire_sefty', 'fire_sensors', 'food_court', 'furnishing', 'internate', 'internet', 'lift', 'meeting_room', 'meeting_rooms', 'metting_room', 'oxygen_duct', 'paarking', 'panetry', 'pantry', 'parking', 'power_backup', 'reception_area', 'security', 'sqcurity', 'staircase', 'staircases', 'staricase', 'vastu', 'water_storae', 'water_storage', 'water_supply']


In [14]:
df.head()

,listing litle,city,area,zone,location_hub,property_type,ownership,size_in_sqft,carpet_area_sqft,private_washroom,public_washroom,floor_no,total_floors,amenities_count,electric_charge_included,water_charge_included,property_age,possession_status,posted_by,lock in period,expected rent increases yearly,negotiable,brokerage,security_deposite,rent_price,floor_0,floor_1,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,paarking,cctv,meeting_rooms,fire_sefty,meeting_room,dg_and_ups,pantry,panetry,reception_area,staircase,cebin,lift,internet,parking,staricase,furnishing,internate,dg,fire_sensors,water_storage,cabin,staircases,fire_extinghuishers,fire_safety,fire_extinguishers,fire_extinguisher,metting_room,fire_safty,water_supply,sqcurity,security,ccvt,vastu,0,water_storae,food_court,oxygen_duct,power_backup
0,showroom for rent,nagpur,dharampeth,east,Retail Complex/ Building,showroom,freehold,900,750,1,1,1 floor,1,"CCTV,food court(2)",no,yes,5 years,ready to move,housing expert,6 months,0.10,yes,yes,450000.0,75000,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,ready to use office space,nagpur,ramdaspeth,east,business park,ready to use office,freehold,600,500,1,1,1 floor,1,"lift, cabin,metting room(3)",no,yes,6 years,ready to move,housing expert,2 months,0.05,yes,yes,0.0,15000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,shop for rent,nagpur,manewada,south,others,shop,freehold,300,250,1,0,ground floor,2,0,no,yes,0,ready to move,owner,12 months,0.10,yes,no,25000.0,12000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,commercial property,nagpur,manewada,south,others,commercial property,freehold,1350,1012,1,1,ground floor,2,0,yes,yes,5 years,ready to move,housing expert,6 months,0.10,yes,yes,40000.0,20000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,ready to use office space,nagpur,manewada,south,others,ready to use office,freehold,2000,1800,1,1,ground floor,2,"water storage, internet,cabin,meeting rooms(4)",yes,yes,10 years,ready to move,owner,6 months,0.00,no,no,200000.0,100000,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
df.to_csv('new_file.csv', index=False) 

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 73 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   listing litle                   1005 non-null   object 
 1   city                            1005 non-null   object 
 2   area                            1005 non-null   object 
 3   zone                            1005 non-null   object 
 4   location_hub                    1005 non-null   object 
 5   property_type                   1005 non-null   object 
 6   ownership                       1005 non-null   object 
 7   size_in_sqft                    1005 non-null   int64  
 8   carpet_area_sqft                1005 non-null   int64  
 9   private_washroom                1005 non-null   int64  
 10  public_washroom                 1005 non-null   int64  
 11  floor_no                        1005 non-null   object 
 12  total_floors                    10

In [17]:
import pandas as pd

# Mapping of duplicates → clean column name
amenities_mapping = {
    'fire_extinghuishers': 'fire_extinguishers',
    'fire_extinguisher': 'fire_extinguishers',
    'staricase': 'staircase',
    'staircases': 'staircase',
    'staircase': 'staircase',
    'ccvt': 'cctv',
    'sqcurity': 'security',
    'internate': 'internet',
    'internate': 'internet',
    'cebin': 'cabin',
    'cebin': 'cabin',
    'panetry': 'pantry',
    'fire_safty': 'fire_safety',
    'fire_sefty': 'fire_safety',
    'water_storae': 'water_storage',
    'meeting_room': 'meeting_rooms',
    'metting_room': 'meeting_rooms',
    'meeting_rooms': 'meeting_rooms',
    'paarking': 'parking'
}

# Loop through mapping and combine columns
for old_col, new_col in amenities_mapping.items():
    if old_col in df.columns:
        if new_col in df.columns:
            df[new_col] = df[new_col] | df[old_col]  # combine using OR for binary
        else:
            df[new_col] = df[old_col]
        df.drop(columns=[old_col], inplace=True)

# Check remaining columns
print(df.columns)


Index(['listing litle', 'city', 'area', 'zone', 'location_hub',
       'property_type', 'ownership', 'size_in_sqft', 'carpet_area_sqft',
       'private_washroom', 'public_washroom', 'floor_no', 'total_floors',
       'amenities_count', 'electric_charge_included', 'water_charge_included',
       'property_age', 'possession_status', 'posted_by', 'lock in period',
       'expected rent increases yearly', 'negotiable', 'brokerage',
       'security_deposite', 'rent_price', 'floor_0', 'floor_1', 'floor_2',
       'floor_3', 'floor_4', 'floor_5', 'floor_6', 'floor_7', 'floor_8',
       'floor_9', 'cctv', 'dg_and_ups', 'pantry', 'reception_area', 'lift',
       'internet', 'parking', 'furnishing', 'dg', 'fire_sensors',
       'water_storage', 'cabin', 'fire_safety', 'fire_extinguishers',
       'water_supply', 'security', 'vastu', '0', 'food_court', 'oxygen_duct',
       'power_backup'],
      dtype='object')


In [18]:

# Define a mapping for similar items
mapping = {
    'Commercial Project': 'commercial project',
    'Business Park': 'business park',
    'Residential Project': 'residential',
    'Retail Complex/ Building': 'retail complex/building',
    'Market/ High Street': 'market/high street',
    'IT Park': 'it park'
    # "others" remains unchanged
}

# Standardize the column values using the mapping
df['location_hub'] = df['location_hub'].replace(mapping)

# Get value counts
counts = df['location_hub'].value_counts()
print(counts)

location_hub
commercial project         320
others                     295
retail complex/building    110
market/high street         100
business park               79
it park                     59
residential                 31
Residential                 11
Name: count, dtype: int64


In [19]:
df['property_age'] = df['property_age'].str.replace(' years', '', regex=False) \
                                       .str.replace(' year', '', regex=False) \
                                       .fillna(0) \
                                       .astype(int)


In [20]:
print(df['property_age'].value_counts())

property_age
5     275
3     108
10    107
2      89
15     88
0      78
4      63
1      58
6      48
20     36
12     32
30     12
7      11
Name: count, dtype: int64


In [21]:
# Clean 'lock in period' column
df['lock_in_period_in_months'] = df['lock in period'].str.replace('months', '', regex=False) \
                                           .str.replace('month', '', regex=False) \
                                           .str.strip() \
                                           .fillna(0) \
                                           .astype(int)

# Check the cleaned values
print(df['lock_in_period_in_months'].value_counts())
# Drop the original column
df = df.drop(columns=['lock in period'])


lock_in_period_in_months
2     261
6     235
12    110
3      88
1      81
11     60
4      59
0      45
10     29
8      20
36     17
Name: count, dtype: int64


In [22]:
df = df.drop('security_deposite', axis=1)

In [23]:
df = df.drop('amenities_count', axis=1)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 54 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   listing litle                   1005 non-null   object 
 1   city                            1005 non-null   object 
 2   area                            1005 non-null   object 
 3   zone                            1005 non-null   object 
 4   location_hub                    1005 non-null   object 
 5   property_type                   1005 non-null   object 
 6   ownership                       1005 non-null   object 
 7   size_in_sqft                    1005 non-null   int64  
 8   carpet_area_sqft                1005 non-null   int64  
 9   private_washroom                1005 non-null   int64  
 10  public_washroom                 1005 non-null   int64  
 11  floor_no                        1005 non-null   object 
 12  total_floors                    10

In [25]:
# Select only object type columns
df_object = df.select_dtypes(include='object')

# Check the columns
print(df_object.columns)
print(df_object.head())


Index(['listing litle', 'city', 'area', 'zone', 'location_hub',
       'property_type', 'ownership', 'floor_no', 'electric_charge_included',
       'water_charge_included', 'possession_status', 'posted_by', 'negotiable',
       'brokerage'],
      dtype='object')
               listing litle    city        area   zone  \
0          showroom for rent  nagpur  dharampeth   east   
1  ready to use office space  nagpur  ramdaspeth   east   
2              shop for rent  nagpur    manewada  south   
3        commercial property  nagpur    manewada  south   
4  ready to use office space  nagpur    manewada  south   

              location_hub        property_type ownership      floor_no  \
0  retail complex/building             showroom  freehold       1 floor   
1            business park  ready to use office  freehold       1 floor   
2                   others                 shop  freehold  ground floor   
3                   others  commercial property  freehold  ground floor   
4     

In [26]:
print("--- Feature Engineering: One-Hot Encoding Categorical Features ---")
categorical_features = ['listing litle', 'city', 'area', 'zone', 'location_hub',
       'property_type', 'ownership', 'floor_no', 'electric_charge_included',
       'water_charge_included', 'possession_status', 'posted_by', 'negotiable',
       'brokerage']
for feature in categorical_features:
    if feature in df.columns:
        df = pd.get_dummies(df, columns=[feature], drop_first=True)
        print(f"One-hot encoded '{feature}'.")
    else:
        print(f"Warning: Categorical feature '{feature}' not found in DataFrame for encoding.")


# Transform 'Rent' using natural logarithm to reduce skewness
print("--- Feature Engineering: Log Transformation of 'Rent' ---")
# Add a small constant to Rent to avoid log(0) if any rent value is 0
df['rent_price'] = np.log1p(df['rent_price']) # Using log1p which is log(1+x)
print("Applied log transformation to 'Rent' column.")

print("\n--- DataFrame after Feature Engineering ---")
print(df.head())
print(df.info())

--- Feature Engineering: One-Hot Encoding Categorical Features ---
One-hot encoded 'listing litle'.
One-hot encoded 'city'.
One-hot encoded 'area'.
One-hot encoded 'zone'.
One-hot encoded 'location_hub'.
One-hot encoded 'property_type'.
One-hot encoded 'ownership'.
One-hot encoded 'floor_no'.
One-hot encoded 'electric_charge_included'.
One-hot encoded 'water_charge_included'.
One-hot encoded 'possession_status'.
One-hot encoded 'posted_by'.
One-hot encoded 'negotiable'.
One-hot encoded 'brokerage'.
--- Feature Engineering: Log Transformation of 'Rent' ---
Applied log transformation to 'Rent' column.

--- DataFrame after Feature Engineering ---
   size_in_sqft  carpet_area_sqft  private_washroom  public_washroom  \
0           900               750                 1                1   
1           600               500                 1                1   
2           300               250                 1                0   
3          1350              1012                 1         

In [27]:
# Define features (X) and target (y)
X = df.drop('rent_price', axis=1)
y = df['rent_price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nData split into training (80%) and testing (20%) sets.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


Data split into training (80%) and testing (20%) sets.
X_train shape: (804, 122), y_train shape: (804,)
X_test shape: (201, 122), y_test shape: (201,)


In [28]:
# Select all numeric columns (int and float)
df_numeric = df.select_dtypes(include=['int64', 'float64'])

# Check the columns
print(df_numeric.columns)
print(df_numeric.head())


Index(['size_in_sqft', 'carpet_area_sqft', 'private_washroom',
       'public_washroom', 'total_floors', 'property_age',
       'expected rent increases yearly', 'rent_price', 'floor_0', 'floor_1',
       'floor_2', 'floor_3', 'floor_4', 'floor_5', 'floor_6', 'floor_7',
       'floor_8', 'floor_9', 'cctv', 'dg_and_ups', 'pantry', 'reception_area',
       'lift', 'internet', 'parking', 'furnishing', 'dg', 'fire_sensors',
       'water_storage', 'cabin', 'fire_safety', 'fire_extinguishers',
       'water_supply', 'security', 'vastu', '0', 'food_court', 'oxygen_duct',
       'power_backup', 'lock_in_period_in_months'],
      dtype='object')
   size_in_sqft  carpet_area_sqft  private_washroom  public_washroom  \
0           900               750                 1                1   
1           600               500                 1                1   
2           300               250                 1                0   
3          1350              1012                 1               

In [29]:
numerical_cols = ['size_in_sqft', 'carpet_area_sqft', 'private_washroom',
       'public_washroom', 'total_floors', 'property_age',
       'expected rent increases yearly', 'rent_price', 'floor_0', 'floor_1',
       'floor_2', 'floor_3', 'floor_4', 'floor_5', 'floor_6', 'floor_7',
       'floor_8', 'floor_9', 'water_supply', 'cabin', 'dg', 'furnishing',
       'water_storage', 'dg_and_ups', 'parking', 'reception_area',
       'food_court', 'cctv', 'fire_extinguishers', '0', 'fire_safety',
       'security', 'fire_sensors', 'vastu', 'pantry', 'oxygen_duct',
       'internet', 'lift', 'power_backup', 'lock_in_period_in_months']
# Filter numerical_cols to only include columns actually present in X_train
numerical_cols_present = [col for col in numerical_cols if col in X_train.columns]

# 1. Replace empty strings with NaN
X_train = X_train.replace(r'^\s*$', np.nan, regex=True)
X_test = X_test.replace(r'^\s*$', np.nan, regex=True)

# 2. Convert all numeric columns to float
X_train[numerical_cols_present] = X_train[numerical_cols_present].apply(pd.to_numeric, errors='coerce')
X_test[numerical_cols_present] = X_test[numerical_cols_present].apply(pd.to_numeric, errors='coerce')

# 3. Fill NaN with median for each column
for col in numerical_cols_present:
    median_val = X_train[col].median()
    X_train[col].fillna(median_val, inplace=True)
    X_test[col].fillna(median_val, inplace=True)

# 4. Scale the numeric columns
scaler = StandardScaler()
X_train[numerical_cols_present] = scaler.fit_transform(X_train[numerical_cols_present])
X_test[numerical_cols_present] = scaler.transform(X_test[numerical_cols_present])

In [30]:
from sklearn.metrics import r2_score, mean_squared_error
print("\n--- Model Training: Random Forest Regressor ---")
# Initialize and train the Random Forest Regressor model
# Using parameters that generally give good results, but further tuning can be done.
rf_model = RandomForestRegressor(n_estimators=450,n_jobs=-1,random_state=42)
rf_model.fit(X_train, y_train)
print("Random Forest Regressor model trained.")

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"\n--- Model Evaluation (Random Forest) ---")
print(f"R-squared (R^2) Score: {r2_rf:.4f}")
print(f"Mean Squared Error (MSE): {mse_rf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.4f}")


--- Model Training: Random Forest Regressor ---
Random Forest Regressor model trained.

--- Model Evaluation (Random Forest) ---
R-squared (R^2) Score: 0.9921
Mean Squared Error (MSE): 0.0069
Root Mean Squared Error (RMSE): 0.0834


In [31]:
import pandas as pd
import numpy as np

# Get feature importances
importances = rf_model.feature_importances_

# If X_train is a DataFrame
feature_names = X_train.columns




# Convert to percentage
importances_percentage = 100 * importances / importances.sum()

# Create DataFrame of feature importances
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance (%)': importances_percentage
}).sort_values(by='Importance (%)', ascending=False)

print(feature_importance_df)


                       Feature  Importance (%)
0                 size_in_sqft    6.158711e+01
99      property_type_showroom    8.149542e+00
1             carpet_area_sqft    5.053444e+00
5                 property_age    4.467740e+00
87  location_hub_business park    2.656227e+00
..                         ...             ...
11                     floor_4    2.267748e-15
16                     floor_9    0.000000e+00
14                     floor_7    0.000000e+00
12                     floor_5    0.000000e+00
13                     floor_6    0.000000e+00

[122 rows x 2 columns]


In [32]:
df.head()

,size_in_sqft,carpet_area_sqft,private_washroom,public_washroom,total_floors,property_age,expected rent increases yearly,rent_price,floor_0,floor_1,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,cctv,dg_and_ups,pantry,reception_area,lift,internet,parking,furnishing,dg,fire_sensors,water_storage,cabin,fire_safety,fire_extinguishers,water_supply,security,vastu,0,food_court,oxygen_duct,power_backup,lock_in_period_in_months,listing litle_commercial property,listing litle_godown for rent,listing litle_ready to use office space,listing litle_shop for rent,listing litle_showroom for rent,listing litle_werehouse,area_babulban,area_bagadganj,area_besa,area_chatrapati nagar,area_civil lines,area_dhantoli,area_dharampeth,area_dighori,area_gadhibag,area_gandhibag,area_ganeshpeth colony,area_hingna,area_hulkeshwar,area_itwari,area_jaitala,area_khamla,area_kharbi,area_lakadganj,area_laxminagar,area_mahal,area_manawada,area_manewada,area_manish nagar,area_mhalgi nagar,area_mihan,area_nandanwan,area_new indora,area_omkar nagar,area_pardi,area_pratap nagar,area_ramdaspeth,area_sadar,area_sakkardara,area_shivaji nagar,area_sitabuldi,area_somalwada,area_swawlambi nagar,area_trimurti nagar,area_wathoda,zone_north,zone_south,zone_west,location_hub_business park,location_hub_commercial project,location_hub_it park,location_hub_market/high street,location_hub_others,location_hub_residential,location_hub_retail complex/building,property_type_commercial property,property_type_godown,property_type_ready to use office,property_type_shop,property_type_shop,property_type_showroom,property_type_werehouse,ownership_freehold,ownership_leasehold,ownership_power_of_attorney,floor_no_1 floor,"floor_no_1, 2,3 floors","floor_no_1,2 floors","floor_no_1,2,3 floors","floor_no_1,2,3,4,GF",floor_no_2 floor,floor_no_3 floor,floor_no_8 floor,floor_no_ground floor,"floor_no_ground floor,1 floor",electric_charge_included_yes,possession_status_ready to move,posted_by_housing expert,posted_by_owner,negotiable_no,negotiable_yes,brokerage_no,brokerage_yes
0,900,750,1,1,1,5,0.10,11.225257,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,6,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,True
1,600,500,1,1,1,6,0.05,9.615872,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,True
2,300,250,1,0,2,0,0.10,9.392745,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,12,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,True,False,True,True,False
3,1350,1012,1,1,2,5,0.10,9.903538,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,6,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [33]:
# --- Saving the trained model and scaler ---
import joblib
print("\n--- Saving Model and Scaler ---")
try:
    joblib.dump(rf_model, 'mc.pkl')
    joblib.dump(scaler, 'sc.pkl')
    joblib.dump(X.columns.tolist(), 'fc.pkl') # Save feature names for consistency
    print("Model, scaler, and feature names saved successfully.")
except Exception as e:
    print(f"Error saving model/scaler: {e}")


--- Saving Model and Scaler ---
Model, scaler, and feature names saved successfully.


In [35]:
import pandas as pd
import numpy as np
# Assuming the model is loaded as 'trained_model'

# List of all columns provided in your request (excluding the target 'rent_price')
FEATURES = [
    'size_in_sqft', 'carpet_area_sqft', 'private_washroom', 'public_washroom',
    'total_floors', 'property_age', 'expected rent increases yearly',
    'fire_extinguishers', 'cabin', 'water_supply', 'dg_and_ups', 'parking',
    'cctv', 'furnishing', 'lift', 'pantry', 'fire_sensors', 'dg', 'vastu',
    'power_backup', 'security', 'food_court', 'fire_safety', 'water_storage',
    'oxygen_duct', 'internet', 'reception_area', 'lock_in_period_in_months',
    # Listing Titles (OHE)
    'listing litle_commercial property', 'listing litle_godown for rent',
    'listing litle_ready to use office space', 'listing litle_shop for rent',
    'listing litle_showroom for rent', 'listing litle_werehouse',
    # Areas (OHE)
    'area_babulban', 'area_bagadganj', 'area_besa', 'area_chatrapati nagar',
    'area_civil lines', 'area_dhantoli', 'area_dharampeth', 'area_dighori',
    'area_gadhibag', 'area_gandhibag', 'area_ganeshpeth colony', 'area_hingna',
    'area_hulkeshwar', 'area_itwari', 'area_jaitala', 'area_khamla',
    'area_kharbi', 'area_lakadganj', 'area_laxminagar', 'area_mahal',
    'area_manawada', 'area_manewada', 'area_manish nagar', 'area_mhalgi nagar',
    'area_mihan', 'area_nandanwan', 'area_new indora', 'area_omkar nagar',
    'area_pardi', 'area_pratap nagar', 'area_ramdaspeth', 'area_sadar',
    'area_sakkardara', 'area_shivaji nagar', 'area_sitabuldi', 'area_somalwada',
    'area_swawlambi nagar', 'area_trimurti nagar', 'area_wathoda',
    # Zones (OHE)
    'zone_north', 'zone_south', 'zone_west',
    # Location Hubs (OHE)
    'location_hub_business park', 'location_hub_commercial project',
    'location_hub_it park', 'location_hub_market/high street',
    'location_hub_others', 'location_hub_residential',
    'location_hub_retail complex/building',
    # Property Types (OHE)
    'property_type_commercial property', 'property_type_godown',
    'property_type_ready to use office', 'property_type_shop',
    'property_type_showroom', 'property_type_werehouse',
    # Ownership (OHE)
    'ownership_freehold', 'ownership_leasehold', 'ownership_power_of_attorney',
    # Floor No (OHE)
    'floor_no_0,1', 'floor_no_0,1,2,3,4', 'floor_no_1', 'floor_no_1,2',
    'floor_no_2', 'floor_no_3', 'floor_no_8',
    # Other binary/categorical (OHE)
    'electric_charge_included_yes', 'possession_status_ready to move',
    'posted_by_housing expert', 'posted_by_owner', 'negotiable_no',
    'negotiable_yes', 'brokerage_no', 'brokerage_yes'
]

# --- USER INPUT SCENARIO ---
# 500 sqft Office space in Civil Lines, 2nd Floor, Ready to Move, 5 years old, Owner Post
user_data = {
    'size_in_sqft': 500.0,
    'carpet_area_sqft': 450.0,
    'private_washroom': 1,
    'public_washroom': 0,
    'total_floors': 5.0,
    'property_age': 5.0,
    'expected rent increases yearly': 0.05,
    'fire_extinguishers': 1, 'cabin': 1, 'water_supply': 1, 'dg_and_ups': 1,
    'parking': 1, 'cctv': 1, 'furnishing': 1, 'lift': 1, 'pantry': 1,
    'fire_sensors': 1, 'dg': 1, 'vastu': 1, 'power_backup': 1, 'security': 1,
    'food_court': 0, 'fire_safety': 1, 'water_storage': 1, 'oxygen_duct': 0,
    'internet': 1, 'reception_area': 1, 'lock_in_period_in_months': 12.0,

    # Listing Title: Ready to Use Office Space
    'listing litle_ready to use office space': 1,

    # Area: Civil Lines
    'area_civil lines': 1,

    # Zone: Needs to be inferred/set based on Civil Lines (e.g., West)
    'zone_west': 1,

    # Location Hub: Market/High Street
    'location_hub_market/high street': 1,

    # Property Type: Ready to Use Office
    'property_type_ready to use office': 1,

    # Ownership: Freehold
    'ownership_freehold': 1,

    # Floor No: 2
    'floor_no_2': 1,

    # Electric Charge Included: No (implicitly 0, if not in the input)
    # Possession Status: Ready to Move
    'possession_status_ready to move': 1,

    # Posted By: Owner
    'posted_by_owner': 1,

    # Negotiable: Yes
    'negotiable_yes': 1,

    # Brokerage: Yes
    'brokerage_yes': 1
}

# 1. Initialize all one-hot encoded features to 0
input_dict = {feature: 0 for feature in FEATURES}

# 2. Update with the specific user data
# Handle numerical features
numerical_features = [
    'size_in_sqft', 'carpet_area_sqft', 'total_floors', 'property_age',
    'expected rent increases yearly', 'lock_in_period_in_months'
]
for feature in numerical_features:
    input_dict[feature] = user_data.get(feature, 0.0) # Use 0.0 as default for safety

# Handle binary/OHE features
binary_ohe_features = [f for f in FEATURES if f not in numerical_features]
for feature in binary_ohe_features:
    input_dict[feature] = user_data.get(feature, 0) # Use 0 as default

# Ensure the final DataFrame/Array has the exact column order
X_test = pd.DataFrame([input_dict])[FEATURES]

# --- PREDICTION STEP (Requires your model) ---

# try:
#     # Assuming 'trained_model' is your loaded model object
#     prediction = trained_model.predict(X_test)
#     print(f"Predicted Rent Price: {prediction[0]:,.2f}")
# except NameError:
#     print("Model object 'trained_model' not provided/loaded.")
# except Exception as e:
#     print(f"An error occurred during prediction: {e}")

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Columns: 110 entries, size_in_sqft to brokerage_yes
dtypes: bool(80), float64(2), int64(28)
memory usage: 314.2 KB


In [40]:
import joblib
import pickle

# Using joblib (commonly used for ML models)
model = joblib.load("mc.pkl")

# or fallback if saved with pickle
# with open("model.pkl", "rb") as f:
#     model = pickle.load(f)

print(type(model))


<class 'sklearn.ensemble._forest.RandomForestRegressor'>


In [41]:
print(model)
print(model.get_params())   # shows hyperparameters


RandomForestRegressor(n_estimators=450, n_jobs=-1, random_state=42)
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 450, 'n_jobs': -1, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

def predict_rent():
    """
    Function to collect user input and predict rent price using the trained model
    """
    
    # Load the saved model, scaler, and feature names
    try:
        model = joblib.load('mc.pkl')
        scaler = joblib.load('sc.pkl')
        feature_names = joblib.load('fc.pkl')
        print("Model and components loaded successfully.")
    except Exception as e:
        print(f"Error loading model components: {e}")
        return
    
    # Collect user input
    print("\n=== Commercial Property Rent Prediction ===")
    print("Please enter the following details:\n")
    
    # Basic property details
    property_type = input("Property Type (showroom/shop/bare shell office/ready to use office/commercial property/werehouse/godown): ")
    size_sqft = float(input("Size in sqft: "))
    carpet_area = float(input("Carpet Area in sqft: "))
    
    # Location details
    area = input("Area (e.g., manewada, jaitala, besa, etc.): ")
    zone = input("Zone (south/west/east/north): ")
    location_hub = input("Location Hub (commercial project/others/retail complex/building/market/high street/business park/it park/residential): ")
    
    # Property features
    private_washroom = int(input("Number of private washrooms: "))
    public_washroom = int(input("Number of public washrooms: "))
    floor_no = input("Floor Number (e.g., ground floor, 1, 2, etc.): ")
    total_floors = int(input("Total floors in building: "))
    
    # Amenities
    amenities_input = input("Amenities (comma-separated, e.g., parking, lift, cctv, power backup): ")
    amenities = [a.strip().lower() for a in amenities_input.split(',')]
    
    # Other details
    electric_charge = input("Electric charge included (yes/no): ")
    water_charge = input("Water charge included (yes/no): ")
    property_age = int(input("Property age in years: "))
    possession_status = input("Possession status (ready to move/under construction): ")
    posted_by = input("Posted by (owner/housing expert/broker): ")
    lock_in_period = int(input("Lock-in period in months: "))
    expected_rent_increase = float(input("Expected yearly rent increase (e.g., 0.05 for 5%): "))
    negotiable = input("Negotiable (yes/no): ")
    brokerage = input("Brokerage (yes/no): ")
    
    # Create a dictionary with user input
    user_data = {
        'listing litle': property_type,
        'city': 'nagpur',  # All data is from Nagpur based on the dataset
        'area': area,
        'zone': zone,
        'location_hub': location_hub,
        'property_type': property_type,
        'ownership': 'freehold',  # Default value as it's the most common
        'size_in_sqft': size_sqft,
        'carpet_area_sqft': carpet_area,
        'private_washroom': private_washroom,
        'public_washroom': public_washroom,
        'floor_no': floor_no,
        'total_floors': total_floors,
        'amenities_count': ', '.join(amenities),
        'electric_charge_included': electric_charge,
        'water_charge_included': water_charge,
        'property_age': property_age,
        'possession_status': possession_status,
        'posted_by': posted_by,
        'lock in period': f"{lock_in_period} months",
        'expected rent increases yearly': expected_rent_increase,
        'negotiable': negotiable,
        'brokerage': brokerage
    }
    
    # Convert to DataFrame
    user_df = pd.DataFrame([user_data])
    
    # Process the data in the same way as training data
    processed_df = preprocess_input(user_df, feature_names, scaler)
    
    # Make prediction
    try:
        prediction_log = model.predict(processed_df)[0]
        # Convert back from log scale
        prediction = np.expm1(prediction_log)
        
        print("\n=== Prediction Result ===")
        print(f"Estimated Rent Price: ₹{prediction:.2f}")
    except Exception as e:
        print(f"Error making prediction: {e}")

def preprocess_input(df, feature_names, scaler):
    """
    Function to preprocess user input in the same way as training data
    """
    # Make a copy to avoid modifying the original
    processed_df = df.copy()
    
    # Process floor_no
    def floor_to_int_list(floor_str):
        floor_str = str(floor_str).lower()
        floor_str = floor_str.replace('floor', '').replace('floors', '').replace(' ', '')
        # Replace ground floor / GF with 0
        floor_str = floor_str.replace('ground', '0').replace('gf', '0')
        # Split multiple floors
        parts = re.split(r'[,]', floor_str)
        floor_numbers = []
        for p in parts:
            try:
                floor_numbers.append(int(p))
            except:
                continue
        # Return as comma-separated string
        if floor_numbers:
            return ','.join(map(str, sorted(floor_numbers)))
        return None
    
    processed_df['floor_no'] = processed_df['floor_no'].apply(floor_to_int_list)
    
    # Process total_floors
    def total_floors_to_int(floor_str):
        try:
            return int(str(floor_str).lower().replace('floors', '').replace('floor', '').strip())
        except:
            return None
    
    processed_df['total_floors'] = processed_df['total_floors'].apply(total_floors_to_int)
    
    # Process size columns
    def size_to_int(size_str):
        try:
            return int(str(size_str).lower().replace('sqft','').replace('sq.ft','').strip())
        except:
            return None
    
    processed_df['size_in_sqft'] = processed_df['size_in_sqft'].apply(size_to_int)
    processed_df['carpet_area_sqft'] = processed_df['carpet_area_sqft'].apply(size_to_int)
    
    # Process amenities
    def extract_amenities_list(text):
        text = str(text).lower()
        # Remove counts in parentheses
        text = re.sub(r'\(\d+\)', '', text)
        # Split by comma and strip spaces
        amenities = [x.strip() for x in text.split(',') if x.strip() != '']
        return amenities
    
    # Create binary columns for amenities
    all_amenities = ['parking', 'vastu', 'lift', 'cabin', 'meeting room', 'dg and ups', 
                    'water storage', 'staircase', 'security', 'cctv', 'power backup', 
                    'reception area', 'pantry', 'fire extinguishers', 'fire safety', 
                    'oxygen duct', 'food court', 'furnishing', 'internet', 'fire sensors']
    
    for amenity in all_amenities:
        amenity_col = amenity.replace(' ', '_')
        processed_df[amenity_col] = processed_df['amenities_count'].apply(
            lambda x: 1 if amenity in extract_amenities_list(x) else 0
        )
    
    # Process property_age
    processed_df['property_age'] = processed_df['property_age'].astype(int)
    
    # Process lock in period
    processed_df['lock_in_period_in_months'] = processed_df['lock in period'].str.replace('months', '', regex=False) \
                                           .str.replace('month', '', regex=False) \
                                           .str.strip() \
                                           .fillna(0) \
                                           .astype(int)
    
    # Standardize location_hub
    mapping = {
        'Commercial Project': 'commercial project',
        'Business Park': 'business park',
        'Residential Project': 'residential',
        'Retail Complex/ Building': 'retail complex/building',
        'Market/ High Street': 'market/high street',
        'IT Park': 'it park'
    }
    processed_df['location_hub'] = processed_df['location_hub'].replace(mapping)
    
    # One-hot encode categorical features
    categorical_features = ['listing litle', 'city', 'area', 'zone', 'location_hub',
                           'property_type', 'ownership', 'floor_no',
                           'electric_charge_included', 'water_charge_included',
                           'possession_status', 'posted_by', 'negotiable', 'brokerage']
    
    for feature in categorical_features:
        if feature in processed_df.columns:
            processed_df = pd.get_dummies(processed_df, columns=[feature])
    
    # Ensure all expected columns are present
    for col in feature_names:
        if col not in processed_df.columns:
            processed_df[col] = 0
    
    # Keep only the columns in the right order
    processed_df = processed_df[feature_names]
    
    # Scale numerical features
    numerical_cols = ['size_in_sqft', 'carpet_area_sqft', 'private_washroom',
                     'public_washroom', 'total_floors', 'property_age',
                     'expected rent increases yearly', 'fire_extinguishers',
                     'food_court', 'cabin', 'lift', '0', 'water_storage', 'dg',
                     'fire_safety', 'security', 'cctv', 'oxygen_duct', 'furnishing', 'vastu',
                     'reception_area', 'internet', 'water_supply', 'fire_sensors',
                     'power_backup', 'dg_and_ups', 'parking', 'pantry',
                     'lock_in_period_in_months']
    
    # Filter to only include columns present in the DataFrame
    numerical_cols_present = [col for col in numerical_cols if col in processed_df.columns]
    
    # Scale the numeric columns
    processed_df[numerical_cols_present] = scaler.transform(processed_df[numerical_cols_present])
    
    return processed_df

# Run the prediction function
if __name__ == "__main__":
    predict_rent()

Model and components loaded successfully.

=== Commercial Property Rent Prediction ===
Please enter the following details:



In [1]:
import pandas as pd
import numpy as np
import joblib
import re
from sklearn.preprocessing import StandardScaler

def display_options(options, title):
    """Display options and get user selection"""
    print(f"\n{title}:")
    for i, option in enumerate(options, 1):
        print(f"{i}. {option}")
    
    while True:
        try:
            choice = int(input(f"Select an option (1-{len(options)}): "))
            if 1 <= choice <= len(options):
                return options[choice-1]
            else:
                print("Invalid choice. Please try again.")
        except ValueError:
            print("Please enter a valid number.")

def predict_rent():
    """
    Function to collect user input and predict rent price using the trained model
    """
    
    # Load the saved model, scaler, and feature names
    try:
        model = joblib.load('mc.pkl')
        scaler = joblib.load('sc.pkl')
        feature_names = joblib.load('fc.pkl')
        print("Model and components loaded successfully.")
    except Exception as e:
        print(f"Error loading model components: {e}")
        return
    
    # Collect user input
    print("\n=== Commercial Property Rent Prediction ===")
    print("Please select from the following options:\n")
    
    # Define options for each field
    property_types = ['showroom', 'shop', 'bare shell office', 'ready to use office', 
                     'commercial property', 'werehouse', 'godown']
    areas = ['manewada', 'jaitala', 'besa', 'omkar nagar', 'itwari', 'hingna', 
            'sitabuldi', 'mahal', 'kharbi', 'mihan', 'pratap nagar', 'ramdaspeth', 
            'dharampeth', 'gandhibag', 'chatrapati nagar', 'nandanwan', 'sadar', 
            'dighori', 'somalwada', 'ganeshpeth colony', 'mhalgi nagar', 'sakkardara', 
            'babulban', 'manish nagar', 'dhantoli', 'khamla', 'laxminagar', 'ajni', 
            'wathoda', 'hulkeshwar', 'pardi', 'new indora', 'civil lines', 'gadhibag', 
            'bagadganj', 'swawlambi nagar', 'manawada', 'trimurti nagar', 'lakadganj', 'shivaji nagar']
    zones = ['south', 'west', 'east', 'north']
    location_hubs = ['commercial project', 'others', 'retail complex/building', 
                    'market/high street', 'business park', 'it park', 'residential']
    ownerships = ['freehold', 'leasehold', 'cooperative society', 'power_of_attorney']
    possession_statuses = ['ready to move', 'Under Construction']
    posted_by_options = ['owner', 'housing expert', 'broker']
    yes_no_options = ['yes', 'no']
    
    # Basic property details
    property_type = display_options(property_types, "Property Type")
    size_sqft = float(input("\nSize in sqft: "))
    carpet_area = float(input("Carpet Area in sqft: "))
    
    # Location details
    area = display_options(areas, "Area")
    zone = display_options(zones, "Zone")
    location_hub = display_options(location_hubs, "Location Hub")
    ownership = display_options(ownerships, "Ownership Type")
    
    # Property features
    private_washroom = int(input("\nNumber of private washrooms: "))
    public_washroom = int(input("Number of public washrooms: "))
    
    floor_options = ['ground floor', '1 floor', '2 floor', '1, 2,3 floors', 
                    'ground floor,1 floor', '1,2,3 floors', '1,2 floors', 
                    '1,2,3,4,GF', '1 , GF floor', '8 floor', '3 floor']
    floor_no = display_options(floor_options, "Floor Number")
    
    total_floors_options = ['3 floors', '1 floor', '2 floors', '4 floors', 
                           '5 floors', '8 floors', '7 floors', '6 floors', 
                           '15 floors', '9 floors', '10 floors']
    total_floors = display_options(total_floors_options, "Total Floors in Building")
    
    # Amenities
    amenities_options = ['parking', 'vastu', 'lift', 'cabin', 'meeting room', 'dg and ups', 
                        'water storage', 'staircase', 'security', 'cctv', 'power backup', 
                        'reception area', 'pantry', 'fire extinguishers', 'fire safety', 
                        'oxygen duct', 'food court', 'furnishing', 'internet', 'fire sensors']
    
    print("\nSelect amenities (enter numbers separated by commas, e.g., 1,3,5):")
    for i, option in enumerate(amenities_options, 1):
        print(f"{i}. {option}")
    
    amenities_input = input("Your selection: ")
    selected_indices = [int(x.strip()) - 1 for x in amenities_input.split(',') if x.strip().isdigit()]
    amenities = [amenities_options[i] for i in selected_indices if 0 <= i < len(amenities_options)]
    
    # Other details
    electric_charge = display_options(yes_no_options, "Electric charge included")
    water_charge = display_options(yes_no_options, "Water charge included")
    property_age = int(input("\nProperty age in years: "))
    possession_status = display_options(possession_statuses, "Possession status")
    posted_by = display_options(posted_by_options, "Posted by")
    
    lock_in_period_options = ['2 months', '6 months', '12 months', '3 months', '1 month', 
                             '11 months', '4 months', '10 months', '6  months', '8  months', 
                             '4  months', '36 months']
    lock_in_period_str = display_options(lock_in_period_options, "Lock-in period")
    lock_in_period = int(re.sub(r'\D', '', lock_in_period_str))
    
    expected_rent_increase_options = ['0.05', '0.10']
    expected_rent_increase_str = display_options(expected_rent_increase_options, "Expected yearly rent increase")
    expected_rent_increase = float(expected_rent_increase_str)
    
    negotiable = display_options(yes_no_options, "Negotiable")
    brokerage = display_options(yes_no_options, "Brokerage")
    
    # Create a dictionary with user input
    user_data = {
        'listing litle': property_type,
        'city': 'nagpur',  # All data is from Nagpur based on the dataset
        'area': area,
        'zone': zone,
        'location_hub': location_hub,
        'property_type': property_type,
        'ownership': ownership,
        'size_in_sqft': size_sqft,
        'carpet_area_sqft': carpet_area,
        'private_washroom': private_washroom,
        'public_washroom': public_washroom,
        'floor_no': floor_no,
        'total_floors': total_floors,
        'amenities_count': ', '.join(amenities),
        'electric_charge_included': electric_charge,
        'water_charge_included': water_charge,
        'property_age': property_age,
        'possession_status': possession_status,
        'posted_by': posted_by,
        'lock in period': f"{lock_in_period} months",
        'expected rent increases yearly': expected_rent_increase,
        'negotiable': negotiable,
        'brokerage': brokerage
    }
    
    # Convert to DataFrame
    user_df = pd.DataFrame([user_data])
    
    # Process the data in the same way as training data
    processed_df = preprocess_input(user_df, feature_names, scaler)
    
    # Make prediction
    try:
        prediction_log = model.predict(processed_df)[0]
        # Convert back from log scale
        prediction = np.expm1(prediction_log)
        
        print("\n=== Prediction Result ===")
        print(f"Estimated Rent Price: ₹{prediction:.2f}")
    except Exception as e:
        print(f"Error making prediction: {e}")

def preprocess_input(df, feature_names, scaler):
    """
    Function to preprocess user input in the same way as training data
    """
    # Make a copy to avoid modifying the original
    processed_df = df.copy()
    
    # Process floor_no
    def floor_to_int_list(floor_str):
        floor_str = str(floor_str).lower()
        floor_str = floor_str.replace('floor', '').replace('floors', '').replace(' ', '')
        # Replace ground floor / GF with 0
        floor_str = floor_str.replace('ground', '0').replace('gf', '0')
        # Split multiple floors
        parts = re.split(r'[,]', floor_str)
        floor_numbers = []
        for p in parts:
            try:
                floor_numbers.append(int(p))
            except:
                continue
        # Return as comma-separated string
        if floor_numbers:
            return ','.join(map(str, sorted(floor_numbers)))
        return None
    
    processed_df['floor_no'] = processed_df['floor_no'].apply(floor_to_int_list)
    
    # Process total_floors
    def total_floors_to_int(floor_str):
        try:
            return int(str(floor_str).lower().replace('floors', '').replace('floor', '').strip())
        except:
            return None
    
    processed_df['total_floors'] = processed_df['total_floors'].apply(total_floors_to_int)
    
    # Process size columns
    def size_to_int(size_str):
        try:
            return int(str(size_str).lower().replace('sqft','').replace('sq.ft','').strip())
        except:
            return None
    
    processed_df['size_in_sqft'] = processed_df['size_in_sqft'].apply(size_to_int)
    processed_df['carpet_area_sqft'] = processed_df['carpet_area_sqft'].apply(size_to_int)
    
    # Process amenities
    def extract_amenities_list(text):
        text = str(text).lower()
        # Remove counts in parentheses
        text = re.sub(r'\(\d+\)', '', text)
        # Split by comma and strip spaces
        amenities = [x.strip() for x in text.split(',') if x.strip() != '']
        return amenities
    
    # Create binary columns for amenities
    all_amenities = ['parking', 'vastu', 'lift', 'cabin', 'meeting room', 'dg and ups', 
                    'water storage', 'staircase', 'security', 'cctv', 'power backup', 
                    'reception area', 'pantry', 'fire extinguishers', 'fire safety', 
                    'oxygen duct', 'food court', 'furnishing', 'internet', 'fire sensors']
    
    for amenity in all_amenities:
        amenity_col = amenity.replace(' ', '_')
        processed_df[amenity_col] = processed_df['amenities_count'].apply(
            lambda x: 1 if amenity in extract_amenities_list(x) else 0
        )
    
    # Process property_age
    processed_df['property_age'] = processed_df['property_age'].astype(int)
    
    # Process lock in period
    processed_df['lock_in_period_in_months'] = processed_df['lock in period'].str.replace('months', '', regex=False) \
                                           .str.replace('month', '', regex=False) \
                                           .str.strip() \
                                           .fillna(0) \
                                           .astype(int)
    
    # Standardize location_hub
    mapping = {
        'Commercial Project': 'commercial project',
        'Business Park': 'business park',
        'Residential Project': 'residential',
        'Retail Complex/ Building': 'retail complex/building',
        'Market/ High Street': 'market/high street',
        'IT Park': 'it park'
    }
    processed_df['location_hub'] = processed_df['location_hub'].replace(mapping)
    
    # One-hot encode categorical features
    categorical_features = ['listing litle', 'city', 'area', 'zone', 'location_hub',
                           'property_type', 'ownership', 'floor_no',
                           'electric_charge_included', 'water_charge_included',
                           'possession_status', 'posted_by', 'negotiable', 'brokerage']
    
    for feature in categorical_features:
        if feature in processed_df.columns:
            processed_df = pd.get_dummies(processed_df, columns=[feature])
    
    # Ensure all expected columns are present
    for col in feature_names:
        if col not in processed_df.columns:
            processed_df[col] = 0
    
    # Keep only the columns in the right order
    processed_df = processed_df[feature_names]
    
    # Scale numerical features
    numerical_cols = ['size_in_sqft', 'carpet_area_sqft', 'private_washroom',
                     'public_washroom', 'total_floors', 'property_age',
                     'expected rent increases yearly', 'fire_extinguishers',
                     'food_court', 'cabin', 'lift', '0', 'water_storage', 'dg',
                     'fire_safety', 'security', 'cctv', 'oxygen_duct', 'furnishing', 'vastu',
                     'reception_area', 'internet', 'water_supply', 'fire_sensors',
                     'power_backup', 'dg_and_ups', 'parking', 'pantry',
                     'lock_in_period_in_months']
    
    # Filter to only include columns present in the DataFrame
    numerical_cols_present = [col for col in numerical_cols if col in processed_df.columns]
    
    # Scale the numeric columns
    processed_df[numerical_cols_present] = scaler.transform(processed_df[numerical_cols_present])
    
    return processed_df

# Run the prediction function
if __name__ == "__main__":
    predict_rent()

Model and components loaded successfully.

=== Commercial Property Rent Prediction ===
Please select from the following options:


Property Type:
1. showroom
2. shop
3. bare shell office
4. ready to use office
5. commercial property
6. werehouse
7. godown


Select an option (1-7):  1

Size in sqft:  1
Carpet Area in sqft:  1



Area:
1. manewada
2. jaitala
3. besa
4. omkar nagar
5. itwari
6. hingna
7. sitabuldi
8. mahal
9. kharbi
10. mihan
11. pratap nagar
12. ramdaspeth
13. dharampeth
14. gandhibag
15. chatrapati nagar
16. nandanwan
17. sadar
18. dighori
19. somalwada
20. ganeshpeth colony
21. mhalgi nagar
22. sakkardara
23. babulban
24. manish nagar
25. dhantoli
26. khamla
27. laxminagar
28. ajni
29. wathoda
30. hulkeshwar
31. pardi
32. new indora
33. civil lines
34. gadhibag
35. bagadganj
36. swawlambi nagar
37. manawada
38. trimurti nagar
39. lakadganj
40. shivaji nagar


Select an option (1-40):  2



Zone:
1. south
2. west
3. east
4. north


...


Brokerage:
1. yes
2. no


Select an option (1-2):  


Please enter a valid number.


Select an option (1-2):  1



=== Prediction Result ===
Estimated Rent Price: ₹77758.50
